### Concatenate all data players from all leagues

In [1]:
import pandas as pd
import glob
import os

# Path to the folder containing the CSV files
folder_path = 'players_data_clean/'

# Get list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read and concatenate all CSV files into a single DataFrame
df_combined = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# Check the resulting DataFrame
df_combined

,Player,Birthdate,League,Club,Footed,Nationality,Position,Minutes,Goals,Assists,...,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials Won,Aerials Lost,% of Aerials Won
0,Amir Rrahmani,1994-02-24,SerieA,Napoli,Right,Kosovo,Center Backs,3406,0.03,0.08,...,1.11,0.42,0.00,0.00,0.00,0.03,3.57,3.28,1.45,69.3
1,Giovanni Di Lorenzo,1993-08-04,SerieA,Napoli,Right,Italy,Fullbacks,3330,0.08,0.05,...,1.24,1.38,0.14,0.00,0.00,0.00,2.54,1.22,1.24,49.5
2,Romelu Lukaku,1993-05-13,SerieA,Napoli,Left,Belgium,Forwards,2843,0.44,0.32,...,1.49,1.14,0.76,0.00,0.00,0.00,1.08,1.33,1.77,42.9
3,Matteo Politano,1993-08-03,SerieA,Napoli,Left,Italy,Att Mid / Wingers,2804,0.10,0.13,...,0.45,1.00,0.19,0.03,0.00,0.00,3.05,0.19,0.64,23.1
4,Scott McTominay,1996-12-08,SerieA,Napoli,Right,Scotland,Midfielders,2938,0.37,0.12,...,1.38,1.84,0.15,0.03,0.00,0.00,4.38,2.02,1.53,56.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,Yukinari Sugawara,2000-06-28,PremierLeague,Southampton,Right,Japan,Fullbacks,1557,0.06,0.06,...,0.87,0.58,0.23,0.00,0.12,0.00,2.54,0.29,0.52,35.7
2433,Adam Armstrong,1997-02-10,PremierLeague,West Bromwich Albion,Right,England,Forwards,1248,0.14,0.14,...,0.87,0.36,0.65,0.00,0.00,0.00,1.66,0.36,0.65,35.7
2434,James Bree,1997-10-11,PremierLeague,Southampton,Right,England,Center Backs,1075,0.00,0.08,...,0.33,0.42,0.00,0.00,0.00,0.00,5.44,1.00,0.59,63.2
2435,Paul Onuachu,1994-05-28,PremierLeague,Southampton,Right,Nigeria,Forwards,1050,0.34,0.09,...,2.40,1.37,0.86,0.00,0.00,0.00,1.63,7.97,5.06,61.2


In [2]:
# Check for NaN values in each column
nan_counts = df_combined.isna().sum()

# Display columns with NaNs and their counts
nan_columns = nan_counts[nan_counts > 0]

if nan_columns.empty:
    print("No NaN values found!")
else:
    print("NaN values detected:")
    print(nan_columns)

No NaN values found!


### Merge clubs' elos for modelling 

In [3]:
elo_df = pd.read_csv("elo_clubs/Elo_Club_VF.csv", sep=';')

In [4]:
from fuzzywuzzy import process

def get_best_match(name, choices):
    result = process.extractOne(name, choices)
    return result[0] 

# Apply fuzzy matching to club names with the same 
df_combined["Club_fuzzy"] = df_combined["Club"].apply(
    lambda x: get_best_match(x, elo_df["Club"].unique())
)

# Merge the matched names
final_df = pd.merge(df_combined, elo_df, left_on='Club_fuzzy', right_on='Club', how='left')
final_df.drop(columns=['Club_fuzzy', 'Club_y'], inplace=True)
final_df.rename(columns={'Club_x': 'Club'}, inplace=True)

In [5]:
final_df

,Player,Birthdate,League,Club,Footed,Nationality,Position,Minutes,Goals,Assists,...,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials Won,Aerials Lost,% of Aerials Won,Elo
0,Amir Rrahmani,1994-02-24,SerieA,Napoli,Right,Kosovo,Center Backs,3406,0.03,0.08,...,0.42,0.00,0.00,0.00,0.03,3.57,3.28,1.45,69.3,1838
1,Giovanni Di Lorenzo,1993-08-04,SerieA,Napoli,Right,Italy,Fullbacks,3330,0.08,0.05,...,1.38,0.14,0.00,0.00,0.00,2.54,1.22,1.24,49.5,1838
2,Romelu Lukaku,1993-05-13,SerieA,Napoli,Left,Belgium,Forwards,2843,0.44,0.32,...,1.14,0.76,0.00,0.00,0.00,1.08,1.33,1.77,42.9,1838
3,Matteo Politano,1993-08-03,SerieA,Napoli,Left,Italy,Att Mid / Wingers,2804,0.10,0.13,...,1.00,0.19,0.03,0.00,0.00,3.05,0.19,0.64,23.1,1838
4,Scott McTominay,1996-12-08,SerieA,Napoli,Right,Scotland,Midfielders,2938,0.37,0.12,...,1.84,0.15,0.03,0.00,0.00,4.38,2.02,1.53,56.9,1838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,Yukinari Sugawara,2000-06-28,PremierLeague,Southampton,Right,Japan,Fullbacks,1557,0.06,0.06,...,0.58,0.23,0.00,0.12,0.00,2.54,0.29,0.52,35.7,1782
2433,Adam Armstrong,1997-02-10,PremierLeague,West Bromwich Albion,Right,England,Forwards,1248,0.14,0.14,...,0.36,0.65,0.00,0.00,0.00,1.66,0.36,0.65,35.7,1538
2434,James Bree,1997-10-11,PremierLeague,Southampton,Right,England,Center Backs,1075,0.00,0.08,...,0.42,0.00,0.00,0.00,0.00,5.44,1.00,0.59,63.2,1782
2435,Paul Onuachu,1994-05-28,PremierLeague,Southampton,Right,Nigeria,Forwards,1050,0.34,0.09,...,1.37,0.86,0.00,0.00,0.00,1.63,7.97,5.06,61.2,1782


In [6]:
final_df_am = final_df[final_df["Position"] == "Att Mid / Wingers"].copy()

In [7]:
final_df_am

,Player,Birthdate,League,Club,Footed,Nationality,Position,Minutes,Goals,Assists,...,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials Won,Aerials Lost,% of Aerials Won,Elo
3,Matteo Politano,1993-08-03,SerieA,Napoli,Left,Italy,Att Mid / Wingers,2804,0.10,0.13,...,1.00,0.19,0.03,0.0,0.0,3.05,0.19,0.64,23.1,1838
9,Khvicha Kvaratskhelia,2001-02-12,SerieA,Paris Saint-Germain,Right,Georgia,Att Mid / Wingers,1178,0.38,0.23,...,2.52,0.00,0.00,0.0,0.0,3.44,0.31,0.38,44.4,1975
10,David Neres,1997-03-03,SerieA,Napoli,Left,Brazil,Att Mid / Wingers,1283,0.14,0.28,...,0.70,0.28,0.00,0.0,0.0,3.65,0.21,0.70,23.1,1838
12,Giacomo Raspadori,2000-02-18,SerieA,Napoli,Right,Italy,Att Mid / Wingers,1103,0.49,0.08,...,1.63,0.08,0.00,0.0,0.0,3.51,0.41,0.90,31.3,1838
14,Michael Folorunsho,1998-02-07,SerieA,Fiorentina,Right,Italy,Att Mid / Wingers,685,0.00,0.00,...,3.15,0.00,0.00,0.0,0.0,4.07,2.37,1.71,58.1,1757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2417,Wes Burns,1994-11-23,PremierLeague,Ipswich Town,Right,Wales,Att Mid / Wingers,926,0.00,0.10,...,1.07,0.00,0.00,0.0,0.0,4.86,0.78,2.04,27.6,1797
2418,Conor Chaplin,1997-02-16,PremierLeague,Ipswich Town,Right,England,Att Mid / Wingers,857,0.11,0.11,...,0.74,0.00,0.00,0.0,0.0,2.84,0.32,0.95,25.0,1797
2419,Nathan Broadhead,1998-04-05,PremierLeague,Ipswich Town,Right,Wales,Att Mid / Wingers,702,0.26,0.00,...,1.67,0.26,0.00,0.0,0.0,4.10,0.13,1.28,9.1,1797
2422,Mateus Fernandes,2004-07-10,PremierLeague,Southampton,Right,Portugal,Att Mid / Wingers,2909,0.06,0.12,...,1.70,0.12,0.00,0.0,0.0,5.54,0.46,0.28,62.5,1782


In [8]:
# Check for NaN values in each column
nan_counts = final_df.isna().sum()

# Display columns with NaNs and their counts
nan_columns = nan_counts[nan_counts > 0]

if nan_columns.empty:
    print("No NaN values found!")
else:
    print("NaN values detected:")
    print(nan_columns)

No NaN values found!


### Running the positions models for all players 

In [9]:
import joblib 

model_paths = {
    "Center Backs": "models/best_model_cb.joblib",
    "Att Mid / Wingers": "models/best_model_am.joblib",
    "Forwards": "models/best_model_fw.joblib",
    "Fullbacks": "models/best_model_fb.joblib",
    "Midfielders": "models/best_model_mid.joblib"
}

models = {pos: joblib.load(path) for pos, path in model_paths.items()}

In [10]:
predicted_chunks = []

for pos, pipeline in models.items():
    df_pos = final_df[final_df["Position"] == pos].copy()
    df_pos["Rating"] = pipeline.predict(df_pos)

    predicted_chunks.append(df_pos)

df_concat = pd.concat(predicted_chunks, ignore_index=True)

In [11]:
# Ensure birthdate is in datetime format
df_concat["Birthdate"] = pd.to_datetime(df_concat["Birthdate"], errors="coerce")

# Compute age as of today
today = pd.Timestamp.today()
df_concat["Age"] = ((today - df_concat["Birthdate"]).dt.days / 365.25).round(0)

### Rating adjustment minutes played 

In [12]:
max_minutes = df_concat["Minutes"].max()

def adjust_rating(row):
    minutes = row["Minutes"]
    rating = row["Rating"]
    
    if pd.isna(minutes):
        return rating  # fallback if minutes missing
    
    if minutes < 1000:
        return rating * (minutes / 1000) ** 0.25
    
    elif minutes > 2000:
        boost_factor = 0.03 * (minutes - 2000) / (max_minutes - 2000)
        return rating * (1 + boost_factor)
    
    else:
        return rating

df_concat["Rating_Adjusted"] = df_concat.apply(adjust_rating, axis=1)


In [13]:
df_concat

,Player,Birthdate,League,Club,Footed,Nationality,Position,Minutes,Goals,Assists,...,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials Won,Aerials Lost,% of Aerials Won,Elo,Rating,Age,Rating_Adjusted
0,Amir Rrahmani,1994-02-24,SerieA,Napoli,Right,Kosovo,Center Backs,3406,0.03,0.08,...,0.00,0.03,3.57,3.28,1.45,69.3,1838,79.483861,31.0,81.050510
1,Alessandro Buongiorno,1999-06-06,SerieA,Napoli,Left,Italy,Center Backs,1924,0.05,0.00,...,0.00,0.00,3.93,2.01,0.98,67.2,1838,77.809277,26.0,77.809277
2,Alessandro Bastoni,1999-04-13,SerieA,Internazionale,Left,Italy,Center Backs,2401,0.04,0.19,...,0.00,0.00,3.60,1.65,0.90,64.7,1933,81.425545,26.0,81.883278
3,Francesco Acerbi,1988-02-10,SerieA,Internazionale,Left,Italy,Center Backs,1710,0.00,0.05,...,0.00,0.00,3.26,3.21,1.58,67.0,1933,79.861999,37.0,79.861999
4,Stefan de Vrij,1992-02-05,SerieA,Internazionale,Right,Netherlands,Center Backs,1726,0.16,0.00,...,0.00,0.00,3.44,3.39,1.98,63.1,1933,80.175203,33.0,80.175203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,Jack Taylor,1998-06-23,PremierLeague,Ipswich Town,Left,Republic of Ireland,Midfielders,860,0.10,0.00,...,0.00,0.00,5.55,0.73,0.84,46.7,1797,75.221898,27.0,72.438408
2433,Flynn Downes,1999-01-20,PremierLeague,Southampton,Right,England,Midfielders,2151,0.04,0.00,...,0.00,0.00,3.47,0.71,0.71,50.0,1782,74.925184,26.0,75.083788
2434,Joe Aribo,1996-07-21,PremierLeague,Southampton,Left,Nigeria,Midfielders,2013,0.13,0.00,...,0.04,0.00,5.01,1.12,1.07,51.0,1782,75.024692,29.0,75.038365
2435,Lesley Ugochukwu,2004-03-26,PremierLeague,Southampton,Right,France,Midfielders,1655,0.05,0.05,...,0.05,0.00,3.64,1.09,0.98,52.6,1782,75.057848,21.0,75.057848


### Potential determinantion 

In [14]:
import numpy as np

# Parameters
reference_elo = df_concat["Elo"].median()   # Centering point for Elo normalization
elo_weight = 0.2      # How strongly Elo affects potential
age_weight = 0.4  # Boost for younger players

# Elo boost relative to reference
df_concat["EloBoost"] = (df_concat["Elo"] - reference_elo) / reference_elo

# Age modifier: favors players under 22
df_concat["AgeModifier"] = (24 / df_concat["Age"]).clip(upper=1.5) ** age_weight

# Potential formula
df_concat["Potential"] = df_concat["Rating_Adjusted"] * (1 + df_concat["EloBoost"] * elo_weight) * df_concat["AgeModifier"]

# Ensure Potential is at least as high as the current adjusted rating
df_concat["Potential"] = np.maximum(df_concat["Potential"], df_concat["Rating_Adjusted"])


In [15]:
df_concat.sort_values("Potential", ascending=False).head(30)

,Player,Birthdate,League,Club,Footed,Nationality,Position,Minutes,Goals,Assists,...,Aerials Won,Aerials Lost,% of Aerials Won,Elo,Rating,Age,Rating_Adjusted,EloBoost,AgeModifier,Potential
612,Lamine Yamal,2007-07-13,LaLiga,Barcelona,Left,Spain,Att Mid / Wingers,2856,0.28,0.41,...,0.03,0.09,25.0,1945,83.178469,18.0,84.176611,0.189602,1.121955,98.023683
751,Désiré Doué,2005-06-03,Ligue1,Paris Saint-Germain,Right,France,Att Mid / Wingers,1730,0.31,0.31,...,0.42,0.52,44.4,1975,83.083282,20.0,83.083282,0.207951,1.075654,93.085714
618,Arda Güler,2005-02-25,LaLiga,Real Madrid,Left,Turkey,Att Mid / Wingers,1250,0.22,0.29,...,0.50,0.79,38.9,1936,83.311959,20.0,83.311959,0.184098,1.075654,92.914401
954,Cole Palmer,2002-05-06,PremierLeague,Chelsea,Left,England,Att Mid / Wingers,3191,0.42,0.23,...,0.06,0.20,22.2,1903,84.997289,23.0,86.416426,0.163914,1.017170,90.781780
947,Bukayo Saka,2001-09-05,PremierLeague,Arsenal,Left,England,Att Mid / Wingers,1729,0.31,0.52,...,0.52,1.04,33.3,1993,86.839198,24.0,86.839198,0.218960,1.000000,90.642065
2010,Jude Bellingham,2003-06-29,LaLiga,Real Madrid,Right,England,Midfielders,2488,0.33,0.29,...,1.09,1.12,49.2,1936,83.776238,22.0,84.349361,0.184098,1.035417,90.552493
1211,Gonçalo Ramos,2001-06-20,Ligue1,Paris Saint-Germain,Right,Portugal,Forwards,1066,0.84,0.25,...,1.69,1.27,57.1,1975,85.746987,24.0,85.746987,0.207951,1.000000,89.313222
941,Mohamed Salah,1992-06-15,PremierLeague,Liverpool,Left,Egypt,Att Mid / Wingers,3371,0.77,0.48,...,0.24,0.37,39.1,1993,87.623776,33.0,89.307872,0.218960,0.880398,89.307872
2248,Ismael Saibari,2001-01-28,Eredivisie,PSV Eindhoven,Right,Morocco,Midfielders,1782,0.56,0.56,...,0.61,0.71,46.2,1797,87.183206,24.0,87.183206,0.099083,1.000000,88.910873
510,Jamal Musiala,2003-02-26,Bundesliga,Bayern Munich,Right,Germany,Att Mid / Wingers,1798,0.60,0.10,...,0.35,0.70,33.3,1919,82.707762,22.0,82.707762,0.173700,1.035417,88.612085


In [16]:
df_concat.drop(columns=["AgeModifier", "EloBoost", "Rating"], inplace=True)

In [17]:
df_concat.rename(columns={"Rating_Adjusted": "Rating"}, inplace=True)

In [18]:
df_concat.to_csv("players_rating_potential_database.csv", index=False)